# Preprocess data with TensorFlow Transform 

credits: [TFX Tutorials Transform](https://www.tensorflow.org/tfx/transform/get_started)

In [2]:
!pip install tensorflow-transform

     |████████████████████████████████| 402 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 75 kB/s  eta 0:00:01
     |████████████████████████████████| 9.5 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 686 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 899 kB/s eta 0:00:01
     |████████████████████████████████| 89 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 879 

     |████████████████████████████████| 169 kB 673 kB/s eta 0:00:01
     |████████████████████████████████| 83 kB 372 kB/s eta 0:00:01
     |████████████████████████████████| 267 kB 425 kB/s eta 0:00:01
     |████████████████████████████████| 144 kB 771 kB/s eta 0:00:01
     |████████████████████████████████| 255 kB 933 kB/s eta 0:00:01
     |████████████████████████████████| 198 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 839 kB/s eta 0:00:01
     |████████████████████████████████| 75 kB 1.2 MB/s eta 0:00:01
  Created wheel for crcmod: filename=crcmod-1.7-cp36-cp36m-linux_x86_64.whl size=35382 sha256=f19060f70e5018624ef512a30aeeb28f71ec58f32ae7f76736f645cde00175e3
  Stored in directory: /root/.cache/pip/wheels/ac/bb/07/adfb4ffd0aaace2022ea25c082a7cdc688b10d30e86d6d2fde
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78532 sha256=533aa5264778a624700de30ea61d8985f98cb39b7e99d776fd62a3b093504c6c
  Stored in directory: /root/.cache/pip/wheel

  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=3fd6450a6cbefaae2e75bffd952a9a930d21ed084d74dc2843af333b95f21a50
  Stored in directory: /root/.cache/pip/wheels/4e/08/0c/727bff8f20fedbdeb8a2c5214e460b214d41c10dc879cf6dac
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=8b476f2020a5c80e226fe9eb28d1f2a36cc3117ff72d8157c60be73b47954fb2
  Stored in directory: /root/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
  Created wheel for google-apitools: filename=google_apitools-0.5.31-py3-none-any.whl size=131041 sha256=1d56ee6bfe9aa61b85dbbbd487a628510baa6443c8f90dad89a33d873a5bbf89
  Stored in directory: /root/.cache/pip/wheels/19/90/12/279bd3b09f3c9a4a0a5416b196eeec18ebc0e11d90ac342159
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=471df53770cecc59e22628c0df244e4dcae17682b21d3311cafa98cccb4e0861
  Stored in directory: /root/.cache/pip

### Preprocessing function 

In [4]:
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

In [6]:
def preprocessing_fn(inputs):
    """Preprocessing function transforms each of three inputs in different ways
    
    Note:
        Input `inputs` must be a dictionary of `Tensor` or `SparseTensor`
    
    Args:
        inputs: Dictionary of `Tensor` or `SparseTensor` of the raw data
    
    Returns:
        A dictionary of `Tensor` or `SparseTensor` containing the transformed values
    """
    x = inputs['x']
    y = inputs['y']
    s = inputs['s']
    x_centered = x - tft.mean(x)
    y_normalized = tft.scale_to_0_1(y)
    s_integerized = tft.compute_and_apply_vocabulary(s)
    x_centered_times_y_normalized = x_centered * y_normalized
    
    return {
        'x_centered': x_centered,
        'y_normalized': y_normalized,
        'x_centered_times_y_normalized': x_centered_times_y_normalized,
        's_integerized': s_integerized
    }

In [11]:
raw_data = [
    {'x': 1, 'y': 1, 's': 'hello'},
    {'x': 2, 'y': 2, 's': 'world'},
    {'x': 3, 'y': 3, 's': 'hello'}
]

In [12]:
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        'x': tf.io.FixedLenFeature([], tf.int64),
        'y': tf.io.FixedLenFeature([], tf.int64),
        's': tf.io.FixedLenFeature([], tf.string),
    })
)

In [17]:
import tensorflow_transform.beam as tft_beam

transformed_dataset, transform_fn = (
(raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
preprocessing_fn))
# transformed_data, transformed_metadata = transformed_dataset

ValueError: A tf.Transform function that required a temp dir was called but no temp dir was set.  To set a temp dir use the impl.Context context manager.

In [10]:
transformed_data

NameError: name 'transformed_data' is not defined